In [17]:
import logging
from typing import List
import multiprocessing

import pandas as pd
import peewee
from playhouse.shortcuts import model_to_dict
from tqdm import tqdm
import gensim

from database import Page

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [2]:
path_data = '/Volumes/ExternalSSD/FakeNewsRecognition/'

In [3]:
Page._meta.database = peewee.SqliteDatabase(path_data + 'news_cleaned_2018_01_29+postgres+nytimes+webhose.db')

In [6]:
def fetch_pages(last_id, batch_size):
    while True:
        pages = list(Page.select()
                     .where((Page.id > last_id) &
                            (peewee.fn.Length(Page.url) > 40) & (peewee.fn.Length(Page.content) > 40))
                     .order_by(Page.id.asc()).limit(batch_size))  # type: List[Page]

        logging.debug('Last id:%s' % (pages[-1].id if len(pages) > 0 else None))
        if len(pages) <= 0:
            return False

        for page in pages:
            yield page

            last_id = page.id

In [27]:
with open(path_data + 'news_cleaned_2018_01_29+postgres+nytimes+webhose.preprocessed.txt', 'w') as out_preprocess:
    with tqdm() as progress: 
        for page in fetch_pages(0, 1000):
            text = page.title + page.content
            text_cleaned = gensim.parsing.preprocessing.preprocess_string(text, [
                gensim.parsing.preprocessing.strip_tags,
                gensim.parsing.preprocessing.strip_punctuation,
                gensim.parsing.preprocessing.strip_multiple_whitespaces,
                gensim.parsing.preprocessing.strip_numeric,
                gensim.parsing.preprocessing.remove_stopwords,
                gensim.parsing.preprocessing.strip_short,
                lambda x: x.lower()
#                 gensim.parsing.preprocessing.stem_text
            ])
            
            out_preprocess.write(' '.join(text_cleaned) + '\n')
            progress.update()

8528956it [3:51:37, 613.71it/s] 
